# Disentangled Multimodal Representation Learning for Recommendation (DMRL)

Ideias:
- FEITO: Usar a coluna para no texto
- FEITO: Limpar os dados de conteudo
- FEITO (lucas): ordenar os itens novos que ficaram por ultimo por popularidade
- FEITO: Adicionar o nome da coluna, porque vai indicar o assunto, em conjunto do texto

- tunar os hiperparametros
- usa a imagem alem do texto
- usar algum algoritmo mais simples apenas para usuarios novos, knn para content based
- Entender o parametro exclude_unknowns=True do RatioSplit e se tem alguma forma do DMRL gerar previsoes para itens e usuarios novos 

- paralelizar a previsao
Perguntas:


## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import cornac
import numpy as np
import pandas as pd
from cornac.metrics import NDCG
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality, ImageModality
from cornac.models.dmrl.recom_dmrl import DMRL 
from tqdm import tqdm
from utils import load_data, preprocessing_content_data
import requests
from PIL import Image
from io import BytesIO
from joblib import Parallel, delayed


# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

/home/matheus/miniconda/envs/sistema_recomendacao_tp2_versao3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and process data

### Load data

In [3]:
ratings, content, targets = load_data()

In [4]:
ratings["TimestampDate"] = ratings['Timestamp'].dt.date
ratings.loc[ratings.Rating == 0, "Rating"] = 0.01

### Limpar os dados

- FEITO - Year: deletar o - no texto '2013–'
- FEITO - Rated: alterar as diversas formas de escrever NA para NA. Esse e um caso especial
- FEITO - alterar as diversas formas de escrever NA para None em todas as coluans
- FEITO - Language: tem varias linhas que possuem bizarices como  'None, English' e 'None, French' , 'English, None'...
- FEITO - Ratings: criar uma coluna para cada chave do dicionario, entender quais sao todas as chaves que existem

-----------------------------------------------
Variaveis qwue nao precisariam ser tratadas com um bert:
- Metascore
- imdbRating
- Type

In [5]:
content_auxiliar = content.drop(columns=["Poster", "Website", "Response", "Episode", "seriesID"]).copy()

In [6]:
content_auxiliar['Year'] = content_auxiliar['Year'].str.replace('–', '')

In [7]:
nan = content.totalSeasons.unique()[0]
dict_transform_to_na = {
    "Rated":['N/A', 'Not Rated', 'Unrated', 'UNRATED', 'NOT RATED'],
    "all": [nan, 'N/A', 'None', np.nan],
}

for na_value in dict_transform_to_na["all"]:
    content_auxiliar = content_auxiliar.replace(na_value, None)

for na_value in dict_transform_to_na["Rated"]:
    content_auxiliar['Rated'] = content_auxiliar['Rated'].replace(na_value, None)

In [8]:
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace('None, ', '')
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace(', None', '')

In [9]:
# Entendendo os valores possiveis para a coluna Ratings
# A coluna content_auxiliar.Ratings quarda uma lista que posde ter entre 0 e 3 dicionarios. Cada dicionario possui a chave 'Source', 'Value'.
num_ratings_per_item = []
unique_keys = []
rating_sources = []
rating_values = []

for rating_list in content_auxiliar.Ratings:
    num_ratings_per_item.append(len(rating_list))
    for rating_dict in rating_list:
        for key in rating_dict:
            unique_keys.append(key)
        rating_sources.append(rating_dict['Source'])
        rating_values.append(rating_dict['Value'])

set(rating_sources)

{'Internet Movie Database', 'Metacritic', 'Rotten Tomatoes'}

In [10]:
InternetMovieDatabase_list = []
Metacritic_list = []
RottenTomatoes_list = []
for rating_list in content_auxiliar.Ratings:
    InternetMovieDatabase_list.append(None)
    Metacritic_list.append(None)
    RottenTomatoes_list.append(None)
    for rating_dict in rating_list:
        if rating_dict['Source'] == 'Internet Movie Database':
            InternetMovieDatabase_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Metacritic':
            Metacritic_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Rotten Tomatoes':
            RottenTomatoes_list[-1] = rating_dict['Value']

In [11]:
content_auxiliar['Internet Movie Database'] = InternetMovieDatabase_list
content_auxiliar['Metacritic'] = Metacritic_list
content_auxiliar['Rotten Tomatoes'] = RottenTomatoes_list


In [12]:
content_auxiliar.drop(columns=['Ratings'], inplace=True)

### Apendar a coluna no valor do dataframe

In [13]:
content_columns = content_auxiliar.columns.to_list()
content_columns.pop(0)

'ItemId'

In [14]:
for column in content_columns:
    content_auxiliar[column] = content_auxiliar[column].apply(lambda x: f"{column}: {x}; " if x is not None else f"{column}: unknown value; ")

In [15]:
content_processed = content_auxiliar[['ItemId']].copy()
content_processed["text"] = content_auxiliar[content_columns].astype(str).fillna('').agg(' '.join, axis=1)

### Pegando os dados de imagem

In [16]:
# folder_path = 'poster_images'

# # Lista todos os arquivos na pasta
# file_list = os.listdir(folder_path)
# item_id_saved = [file.split('=')[1].split('.')[0] for file in file_list]

In [17]:
# content_to_get_image = content.loc[content.Poster != 'N/A', ['ItemId', 'Poster']].copy()
# content_to_get_image = content_to_get_image.loc[~content_to_get_image.ItemId.isin(item_id_saved), :]
# content_to_get_image = content_to_get_image.reset_index(drop=True)

In [18]:
# def download_image(index):
#     image_url = content_to_get_image.loc[index, "Poster"]
#     item_id = content_to_get_image.loc[index, "ItemId"]

#     # Fazer a requisição HTTP para baixar a imagem
#     # try:
#     response = requests.get(image_url)
#     # except:
#         # return None

#     # Verificar se a requisição foi bem-sucedida
#     if response.status_code == 200:
#         # Abrir a imagem
#         img = Image.open(BytesIO(response.content))
#         # Salvar a imagem
#         img.save(f"{folder_path}/item_id={item_id}.jpg")
#         return None
#     else:
#         return (item_id, image_url)

# # Lista de índices para processar
# indices = content_to_get_image.index.tolist()

# # Usar todos os CPUs disponíveis
# result_list = Parallel(n_jobs=-1, verbose=100)(delayed(download_image)(index) for index in indices)

# # Filtrar os resultados que não conseguiram baixar
# nao_conseguiu_baixar = [result for result in result_list if result is None]

## Train model

In [19]:
# from PIL import Image
# import numpy as np
# import os

# folder_path = 'poster_images'
# image_files = os.listdir(folder_path)
# images_item_ids = [file.split('=')[1].split('.')[0] for file in image_files]

# # Initialize a list to store the image matrices
# def process_image(image_file):
#     image_path = os.path.join(folder_path, image_file)
#     image = Image.open(image_path)
#     image_matrix = np.array(image)
#     return image_matrix.reshape(-1)

# # Use Parallel to process images in parallel
# image_matrices = Parallel(n_jobs=-1, verbose=100)(delayed(process_image)(image_file) for image_file in image_files)

In [20]:
#  E preciso que todas as imagens tenham o mesmo tamanho, para isso vamos adicionar zeros no final das imagens menores

# maior_imagem = -1
# for image in image_matrices:
#     if image.shape[0] > maior_imagem:
#         maior_imagem = image.shape[0]

# for index, image in enumerate(image_matrices):
#     new_image = np.insert(image, len(image), np.zeros(maior_imagem - len(image)))
#     del image
#     image_matrices[index] = new_image

# def add_zeros_to_image(image):
#     new_image = np.insert(image, len(image), np.zeros(maior_imagem - len(image)))
#     del image
#     return new_image

# image_matrices = Parallel(n_jobs=-1, verbose=100)(delayed(add_zeros_to_image)(image) for image in image_matrices)

In [21]:
#  E preciso que todas as imagens tenham o mesmo tamanho, para isso vamos deletar as maoires imagens
# menor_imagem = np.Inf
# for image in image_matrices:
#     if image.shape[0] < menor_imagem:
#         menor_imagem = image.shape[0]

# def diminui_a_imagem(image):
#     return image[:menor_imagem]

# image_matrices2 = Parallel(n_jobs=-1, verbose=100)(delayed(diminui_a_imagem)(image) for image in image_matrices)

# Convert the list of image matrices to a numpy array
# image_matrices2 = np.array(image_matrices2)
# image_modality = ImageModality(features=image_matrices2, ids=images_item_ids)

In [22]:
item_text_modality = TextModality(
    corpus=content_processed.text.to_list(),
    ids=content_processed.ItemId.to_list(),
)

In [23]:
from skopt import dummy_minimize

In [24]:
# conda install pytorch torchvision torchaudio cudatoolkit=11.7 -c pytorch -c nvidia

In [31]:
import dill

def save_object(obj, filename):
    with open(filename, 'wb') as file:
        dill.dump(obj, file)

def load_object(filename):
    with open(filename, 'rb') as file:
        return dill.load(file)

In [32]:
def DMR_pipeline(parameters):
    learning_rate, decay_c, decay_r, num_factors = parameters
    # learning_rate, decay_c, decay_r, epochs, embedding_dim, bert_text_dim, num_factors = parameters
    print("Parametros da minimizacao" + "="*150)
    model_parameters = (
        # f"learning_rate={learning_rate},decay_c={decay_c},decay_r={decay_r},embedding_dim={embedding_dim},bert_text_dim={bert_text_dim},num_factor={num_factors}"
        f"learning_rate={learning_rate},decay_c={decay_c},decay_r={decay_r},num_factor={num_factors}"

        )
    print(model_parameters)

    ratio_split = RatioSplit(
        data=ratings[['UserId', 'ItemId', 'Rating']].values.tolist(),
        test_size=0.2,
        exclude_unknowns=True,
        verbose=True,
        seed=12012001,
        rating_threshold=0.5,
        item_text=item_text_modality
    )
    dmrl_recommender = DMRL(
        batch_size=1024,
        epochs=20,
        learning_rate=learning_rate,
        decay_c=decay_c,
        decay_r=decay_r,
        # epochs=epochs,
        # embedding_dim=embedding_dim,
        # bert_text_dim=bert_text_dim,
        num_factors=num_factors,
        verbose=True,
    )
    experiment = cornac.Experiment(
        eval_method=ratio_split,
        models=[dmrl_recommender], metrics=[NDCG(20)]
    )
    
    try:
        print("Iniciando o experimento...")
        experiment_returned = experiment.run()
        metric = experiment.result[0].metric_avg_results['NDCG@20']
        print("Metrica NDCG@20: ", metric)
        model_path = "dmrl_models/" + f"NDCG@20={metric}," + model_parameters + ".model"
        save_object(dmrl_recommender, model_path)
        # dmrl_recommender.save("dmrl_models/" + f"NDCG@20={metric}," + model_parameters + ".model")
        print("Fim do experimento", "="*50)
        return -1 * metric
    except RuntimeError as e:
        print("Erro na execução do experimento, retornando Infinito")
        print(e)
        print("Fim do experimento", "="*100)
        return np.Inf

In [33]:
space = [
    (0.00001, 0.1),  # learning_rate
    (0.001, 1),  # decay_c # comentar isso nao resolveu o problema do cuda
    (0.0001, 0.1),  # decay_r # comentar isso nao resolveu o problema do cuda
    # [20, 30, 40, 50], # epochs # comentar isso nao resolveu o problema do cuda
    # [40, 128, 200, 300, 500],  # embedding_dim # tem relacao com o erro de dimensao de multiplicacao de matriz
    # [100, 383, 600, 800, 1000],  # bert_text_dim # tem relacao com o erro de dimensao de multiplicacao de matriz
    [2, 4, 8],  # num_factors # comentar isso nao resolveu o problema do cuda
]

In [ ]:
result = dummy_minimize(DMR_pipeline, space, verbose=True, n_calls=30, random_state=12012001)

Iteration No: 1 started. Evaluating function at random point.
Parametros da minimizacao======================================================================================================================================================
learning_rate=0.02304042515912769,decay_c=0.6461142798196924,decay_r=0.0456619901461552,num_factor=4
rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 46750
Number of items = 27045
Number of ratings = 527776
Max rating = 10.0
Min rating = 0.0
Global mean = 7.3
---
Test data:
Number of users = 46750
Number of items = 27045
Number of ratings = 124031
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 46750
Total items = 27045
Iniciando o experimento...

[DMRL] Training started!


/home/matheus/miniconda/envs/sistema_recomendacao_tp2_versao3/lib/python3.9/site-packages/cornac/models/dmrl/transformer_text.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

Using device cuda for training
Erro na execução do experimento, retornando Infinito
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!
Fim do experimento ====================================================================================================
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 23.4232
Function value obtained: inf
Current minimum: inf
Iteration No: 2 started. Evaluating function at random point.
Parametros da minimizacao======================================================================================================================================================
learning_rate=0.09938198721604344,decay_c=0.8755381083510204,decay_r=0.0004231976548177414,num_factor=2
rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 46750
Number of items = 27045
Number of ratings = 527776
Max rating = 10.0
Min rating = 0.0
Global mean = 7.3
---
Test data:
Number of users = 46750


/home/matheus/miniconda/envs/sistema_recomendacao_tp2_versao3/lib/python3.9/site-packages/cornac/models/dmrl/transformer_text.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

Using device cuda for training
  batch 5 loss: 714.6182983398437
  batch 10 loss: 731.3954223632812
  batch 15 loss: 732.7428466796875
  batch 20 loss: 726.7514892578125
  batch 25 loss: 705.0027099609375
  batch 30 loss: 700.1363647460937
  batch 35 loss: 689.5919799804688
  batch 40 loss: 682.445361328125
  batch 45 loss: 687.4040283203125
  batch 50 loss: 687.7245239257812
  batch 55 loss: 663.6460571289062
  batch 60 loss: 661.2365478515625
  batch 65 loss: 663.7310180664062
  batch 70 loss: 653.9932739257813
  batch 75 loss: 648.060205078125
  batch 80 loss: 657.8062866210937
  batch 85 loss: 635.0588500976562
  batch 90 loss: 636.6204467773438
  batch 95 loss: 640.6646606445313
  batch 100 loss: 634.740478515625
  batch 105 loss: 631.378466796875
  batch 110 loss: 643.8226806640625
  batch 115 loss: 632.57548828125
  batch 120 loss: 638.30078125
  batch 125 loss: 634.914013671875
  batch 130 loss: 646.8117431640625
  batch 135 loss: 626.494140625
  batch 140 loss: 633.32216796875

...

/home/matheus/miniconda/envs/sistema_recomendacao_tp2_versao3/lib/python3.9/site-packages/cornac/models/dmrl/transformer_text.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

Using device cuda for training
Erro na execução do experimento, retornando Infinito
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!
Fim do experimento ====================================================================================================
Iteration No: 13 ended. Evaluation done at random point.
Time taken: 14.1359
Function value obtained: inf
Current minimum: -0.0849
Iteration No: 14 started. Evaluating function at random point.
Parametros da minimizacao======================================================================================================================================================
learning_rate=0.09976504794291016,decay_c=0.12555419371606055,decay_r=0.007040115440468155,num_factor=2
rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 46750
Number of items = 27045
Number of ratings = 527776
Max rating = 10.0
Min rating = 0.0
Global mean = 7.3
---
Test data:
Number of users = 

/home/matheus/miniconda/envs/sistema_recomendacao_tp2_versao3/lib/python3.9/site-packages/cornac/models/dmrl/transformer_text.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

Using device cuda for training
  batch 5 loss: 713.57080078125
  batch 10 loss: 736.5335083007812
  batch 15 loss: 777.249072265625
  batch 20 loss: 810.6352783203125
  batch 25 loss: 805.11591796875
  batch 30 loss: 759.1670776367188
  batch 35 loss: 711.9390991210937
  batch 40 loss: 695.0619750976563
  batch 45 loss: 693.1284912109375
  batch 50 loss: 683.3386596679687
  batch 55 loss: 666.6586547851563
  batch 60 loss: 670.98662109375
  batch 65 loss: 657.824365234375
  batch 70 loss: 652.1649169921875
  batch 75 loss: 641.4106811523437
  batch 80 loss: 629.3401123046875
  batch 85 loss: 634.9309936523438
  batch 90 loss: 620.8404418945313
  batch 95 loss: 605.3079711914063
  batch 100 loss: 595.0427734375
  batch 105 loss: 600.9548706054687
  batch 110 loss: 588.3468017578125
  batch 115 loss: 597.0385009765625
  batch 120 loss: 580.94814453125
  batch 125 loss: 582.5617065429688
  batch 130 loss: 579.6213989257812
  batch 135 loss: 566.9156494140625
  batch 140 loss: 574.31667480

Ranking:  96%|█████████▌| 19100/19975 [04:27<00:11, 75.85it/s]

In [ ]:
# def compute_score(user_id):
#     target_line = pd.DataFrame()
#     user_index = ratio_split.train_set.uid_map.get(user_id)

#     columns = ["UserId", "ItemId", "Rating"]
#     items_to_predict = targets.loc[targets.UserId == user_id, "ItemId"].to_list()
#     target_line["ItemId"] = items_to_predict
#     target_line["UserId"] = user_id
#     target_line["Rating"] = -1
#     if user_index is None:
#         return target_line[columns]

#     # Get the train dataframe index of the items to predict
#     items_to_predict_index = np.array([ratio_split.train_set.iid_map.get(item_id) for item_id in items_to_predict])
#     items_to_predict_tensor = torch.tensor([idx for idx in items_to_predict_index if idx is not None])

#     # Get the position of items that are not in the train set
#     none_indices = [i for i, x in enumerate(items_to_predict_index) if x is None]

#     # Get the prediction for the items
#     line_rating = dmrl_recommender.score(user_index=user_index, item_indices=items_to_predict_tensor)

#     # Insert -1 in the position of items that are not in the train set
#     for index_to_insert in none_indices:
#         line_rating = np.insert(line_rating, index_to_insert, -1)

#     # Insert the prediction in the target_prediction dataframe
#     target_line["Rating"] = line_rating
#     return target_line[columns]

# user_id_list = targets.UserId.unique()
# target_prediction_list = Parallel(n_jobs=-1, verbose=100)(delayed(compute_score)(user_id) for user_id in user_id_list)
# target_prediction = pd.concat(target_prediction_list, ignore_index=True)

In [ ]:
target_prediction = targets.copy()
target_prediction["Rating"] = -1

user_id_list = targets.UserId.unique()
for user_id in user_id_list:
    # Get the train dataframe index of the user to predict
    user_index = ratio_split.train_set.uid_map.get(user_id)

    if user_index is None:
        print(f"User {user_id} is not in the train set")
        continue

    # Flter by items to predict 
    items_to_predict = targets.loc[targets.UserId == user_id, "ItemId"].to_list()

    # Get the train dataframe index of the items to predict
    items_to_predict_index = np.array([ratio_split.train_set.iid_map.get(item_id) for item_id in items_to_predict])

    items_to_predict_tensor = torch.tensor([idx for idx in items_to_predict_index if idx is not None])

    # Get the position of items that are not in the train set
    none_indices = [i for i, x in enumerate(items_to_predict_index) if x is None]

    # Get the prediction for the items
    line_rating = dmrl_recommender.score(user_index=user_index, item_indices=items_to_predict_tensor)

    # Insert -1 in the position of items that are not in the train set
    for index_to_insert in none_indices:
        line_rating = np.insert(line_rating, index_to_insert, -1)

    # Insert the prediction in the target_prediction dataframe
    target_prediction.loc[targets.UserId == user_id, "Rating"] = line_rating

In [33]:
target_prediction = target_prediction.sort_values(["UserId", "Rating"], ascending=[True, False])

In [34]:
target_prediction.to_csv("submissao_5_DMRL_versao_4.csv", index=False)

In [35]:
target_prediction = target_prediction.drop(columns="Rating")

In [36]:
target_prediction.to_csv("submissao_5_DMRL_versao_4_sem_rating.csv", index=False)